In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp/BUDGET.txt"))

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/emp"))

In [0]:
dbutils.notebook.run("/https://dbc-7ce32fe8-d0a3.cloud.databricks.com/editor/notebooks/3296070990322407?o=1322396891375465", 60)

In [0]:
dbutils.fs.mv("/Volumes/workspace/default/emp/BUDGET", "/Volumes/workspace/default/emp/BUDGET.txt")

In [0]:
dbutils.fs.cp("/Volumes/workspace/default/emp/BUDGET.txt", "/Volumes/workspace/default/emp/BUDGET_copy")

In [0]:
dbutils.fs.put("/Volumes/workspace/default/emp/BUDGET_test", "This is a test file")

In [0]:
dbutils.fs.rm("/Volumes/workspace/default/emp/BUDGET_test")

In [0]:
display(dbutils.fs.head("/Volumes/workspace/default/emp/BUDGET_copy"))

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/default/emp/emp_CSV_salt")
dbutils.fs.cp("/Volumes/workspace/default/emp/BUDGET_copy", "/Volumes/workspace/default/emp/emp_CSV_salt")

In [0]:
dbutils.fs.rm("/Volumes/workspace/default/emp/emp_CSV_salt", recurse=True)

In [0]:
dbutils.fs.mount("dbfs:/Volumes/workspace/default/emp/emp_CSV_salt", "/mnt/emp/emp_CSV_salt")

In [0]:
dbutils.fs.unmount("dbfs:/Volumes/workspace/default/emp/emp_CSV_salt", "/mnt/emp/emp_CSV_salt")

In [0]:
dbutils.fs.refreshMounts("dbfs:/Volumes/workspace/default/emp/emp_CSV_salt", "/mnt/emp/emp_CSV_salt")

**NOTEBOOK**

In [0]:
dbutils.notebook.run("DBNOTEBOOK_TEST", 60)

In [0]:
dbutils.notebook.exit("DBNOTEBOOK_TEST")

**Secrets**

In [0]:
dbutils.secrets.get(scope="dbfs", key="dbfs")
dbutils.widgets.get("DBWIDGET_TEST")

In [0]:
dbutils.secrets.list(scope="dbfs")

In [0]:
dbutils.secrets.getbytes(scope="dbfs", key="dbfs")

In [0]:
dbutils.secrets.listScopes()

**Widgets**

In [0]:
dbutils.widgets.text("folder_name" ,"","")
dbutils.widgets.text("file_name" ,"","")

In [0]:
dbutils.widgets.get("DBWIDGET_TEST")

In [0]:
folder_loc = dbutils.widgets.get("folder_name")
file_loc = dbutils.widgets.get("file_name")

print("folder var is: ", folder_loc)
print("file var is: ", file_loc)


In [0]:
dbutils.widgets.dropdown("dropdown", "1", [str(x) for x in range(1, 10)])

In [0]:
dbutils.widgets.combobox("combobox",  "1", [str(x) for x in range(1, 10)])

In [0]:
dbutils.widgets.multiselect("Policy", "vehicle",("vehicle","bike","car","truck"))

**remove**

In [0]:
dbutils.widgets.remove("DBWIDGET_TEST")

In [0]:
dbutils.widgets.removeAll()

In [0]:
%run '/Users/suganthibala2701@gmail.com/DBNOTEBOOK_TEST'

**EXPLODE**

In [0]:
#Explode - when array  is passed to this function, it converts into rows


In [0]:
/Workspace/Users/suganthibala2701@gmail.com/DBNOTEBOOK_TEST

In [0]:
arr_appl = [('Hari',['TV','AC','Refrigerator','Washing Machine']),
            ('Kanish',['AC','Refrigerator','Washing Machine']),
            ('Bala',['Refrigerator','Washing Machine']),
            ('Sugan',['Refrigerator', 'grinder','Machine'])]
df_app = spark.createDataFrame(arr_appl,['name','appliances'])
df_app.display()


In [0]:
from pyspark.sql.functions import explode, col, split, collect_list

df_exp = df_app.withColumn('appliances', explode(col('appliances')))
df_exp.display()

In [0]:
df_splt = df_exp.withColumn('appliances', split(col('appliances'),',')).display()
df_splt.groupBy("name").agg(collect_list("appliances")).display()

In [0]:
df_exp = df_exp.withColumn('appliances', split(col('appliances'), ','))
df_exp.groupBy("name").agg({"appliances": "collect_list"}).display()

In [0]:
df_exp.groupBy("name").agg(collect_list("appliances").alias("appliances_list")).display()

**MAP**

In [0]:
map_brand = [('Hari',{'TV':'Samsung','AC':'LG','Refrigerator':'Whirlpool','Washing Machine':'Bosch'}),
('Kanish',{'AC':'LG','Refrigerator':'Whirlpool','Washing Machine':'Bosch'}),
('Bala',{'Refrigerator':'Whirlpool','Washing Machine':'Bosch'}),
('Sugan',None)]
df_brand = spark.createDataFrame(map_brand,['name','brand'])
df_brand.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

display(df_app.select(df_app.name, explode_outer(df_app.appliances)))

# Null value also incled wit explode_outer
display(df_brand.select(df_brand.name, explode_outer(df_brand.brand)))

**POSEEXPLODE**

In [0]:
#positional explode ignore null values
display(df_app.select(df_app.name, posexplode(df_app.appliances)))
display(df_brand.select(df_brand.name, posexplode(df_brand.brand)))

In [0]:
#posexplode_outer - consider null values

df_brand.select(df_brand.name, posexplode_outer(df_brand.brand)).display()

**PIVOT**

In [0]:
# PIVOT is used to transpose list of values of a columns to columns
#UNPIVOT is opposite of PIVOT,  transpose columns into list of values to a column

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_budget = spark.read.format("csv").option("header", "true").load("/Volumes/workspace/default/emp/BUDGET.txt")
df_budget.display()


In [0]:
df_unpivot = df_budget.selectExpr("YEAR","ACCOUNT","stack(12,'MONTH1',MONTH1,'MONTH2',MONTH2,'MONTH3',MONTH3,'MONTH4',MONTH4,'MONTH5',MONTH5,'MONTH6',MONTH6,'MONTH7',MONTH7,'MONTH8',MONTH8,'MONTH9',MONTH9,'MONTH10',MONTH10,'MONTH11',MONTH11,'MONTH12',MONTH12) AS (MONTH,AMOUNT)")
df_unpivot.display()
#df_unpivot.withColumn(col("AMOUNT").cast("int")).display()
 

In [0]:
df_unpivot = df_unpivot.withColumn("AMOUNT", col("AMOUNT").cast("int"))
display(df_unpivot)

In [0]:
df_unpivot.write.format("delta").mode("overwrite").saveAsTable("budget_unpivot")
df_pivot.write.format("delta").mode("overwrite").saveAsTable("budget_pivot")

In [0]:
%sql
select * from budget_unpivot;

In [0]:
df_pivot = df_unpivot.groupBy("YEAR","ACCOUNT").pivot("MONTH").sum("AMOUNT")
df_pivot.display()

In [0]:
df_country = spark.read.format("csv").option("header", "true").load("/Volumes/workspace/default/emp/HR_COUNTRIES_NEW.csv")
df_country.display()

In [0]:
df_cn_spl = df_country.withColumn('first_name', split(df_country['COUNTRY_NAME'], ' ').getItem(0))\
    .withColumn('last_name', split(df_country['COUNTRY_NAME'], ' ').getItem(1))

df_cn_spl.display()